<a href="https://colab.research.google.com/github/yubraaj11/sentence_transformer_nepali/blob/master/Recommending_using_trained_sentence_transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Mon Jun  5 07:10:02 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   65C    P8    11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Defining the path for the trained and saved SentenceTransformer model to produce encodings of the scraped dataset.

In [3]:
model_path = '/content/drive/My Drive/trained_Model/sentence_transformer_nepali' 
csv_path = '/content/drive/MyDrive/Datasets/3k_News.csv'

In [4]:
import pandas as pd

In [19]:
df = pd.read_csv(csv_path, index_col=0)
df

,id,title,article,link
0,0,"काँक्राका फाइदै-फाइदा, कुन समयमा खानु राम्रो ?",चर्को गर्मीमा काँक्रा खानुको मज्जा नै बेग्लै ...,https://www.onlinekhabar.com/2023/05/1312396
1,1,"थाइराइडका बिरामीले के खाने, के नखाने ?",काठमाडौं । शरीरलाई राम्रोसँग काम गर्न विभिन्न...,https://www.onlinekhabar.com/2023/05/1312323
2,2,बच्चामा अन्धोपनको जोखिम बढाउने आरओपी समस्या के...,सामान्य बच्चाको तुलनामा समयअगावै जन्मिएका बच्...,https://www.onlinekhabar.com/2023/05/1312266
3,3,फुङलिङमा सवारी दुर्घटनामा परी एक बालककाे मृत्यु,"१२ जेठ, ताप्लेजुङ। स्कार्पियाे दुर्घटनामा बिह...",https://www.onlinekhabar.com/2023/05/1312637
4,4,जोर्डनका युवराज र साउदी युवतीबीचको विवाह किन छ...,"१२ जेठ, काठमाडौं । जोर्डनका युवराज हुसेन बिन ...",https://www.onlinekhabar.com/2023/05/1312564
...,...,...,...,...
3852,3852,ज्येष्ठ सदस्य जबरासहित ११ सांसदले बुझाएनन् सम्...,"१५ फागुन, काठमाडौं । प्रतिनिधिसभाका ११ सदस्यल...",https://www.onlinekhabar.com/2023/02/1269914
3853,3853,गुल्मीमा बिभिन्न कार्यक्रम गरेर ४१ औं मगर दिवस...,"१५ फागुन, गुल्मी । गुल्मी जिल्ला सदरमुकाम तम्...",https://www.onlinekhabar.com/2023/02/1269908
3854,3854,कास्कीमा ६ महिनामै बलात्कारका ३५ उजुरी,"१५ फागुन, पोखरा । पोखराको लेकसाइड, शान्तिनगरब...",https://www.onlinekhabar.com/2023/02/1269895
3855,3855,प्रज्ञा प्रतिष्ठानका सदस्यले दोहोरो सुविधा नपाउने,"१५ फागुन, काठमाडौं । नेपाल प्रज्ञा प्रतिष्ठान...",https://www.onlinekhabar.com/2023/02/1269881


In [6]:
import torch 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [7]:
!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 85.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 87.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 118.6 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=1e2782dfecea84ab161a69e6c27c2aa8d5f446786217c3d9a8bdb35bece51686
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


In [8]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer(model_path)
model.to(device)
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

In [20]:
embeddings = model.encode(df['article'])
embeddings.shape

(3857, 768)

In [21]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_similarities = cosine_similarity(embeddings, embeddings)

cosine_similarities.shape

(3857, 3857)

In [22]:
results = {}
for idx, row in df.iterrows():
    similar_indices = cosine_similarities[idx].argsort()[:-100:-1]
    similar_items = [(cosine_similarities[idx][i], df['id'][i]) for i in similar_indices]
    results[row['id']] = similar_items[1:]
print('done!')

done!


In [23]:
similar_indices[:10]

array([3856, 2090,  970,  998, 2451, 3485,   76,  667, 1058, 1663])

In [24]:
def item(id):
    return df.loc[df['id'] == id]['title'].tolist()[0].split(' - ')[0]

# Just reads the results out of the dictionary.
def recommend(item_id, num):
    print("Recommending " + str(num) + " products similar to " + item(item_id) + "...")
    print("-------")
    recs = results[item_id][:num]
    for rec in recs:
        print("Recommended : " + item(rec[1]) + " (score:" + str(rec[0]) + ")",end='\n\n')

recommend(item_id=10, num=10)

Recommending 10 products similar to जनकपुर ११ का वडाध्यक्ष र वडासचिवविरुद्ध भ्रष्टाचारको मुद्दा...
-------
Recommended : शुद्धोधन-७ का वडा अध्यक्ष ५० हजार घुससहित पक्राउ (score:0.8842877)

Recommended : नक्कली परीक्षार्थी राखेर एसईई दिएको आरोपमा मेयरविरुद्ध पक्राउ पुर्जी (score:0.88426626)

Recommended : वृद्धभत्ताको रकम हिनामिनाको आरोपमा वडासचिव विरुद्ध भ्रष्टाचार मुद्दा (score:0.87886274)

Recommended : सप्तरीको शम्भुनाथ नगरपालिकाका मेयरविरुद्ध भ्रष्टाचार मुद्दा (score:0.8761473)

Recommended : नक्कली भुटानी शरणार्थी प्रकरणको अनुसन्धान प्रतिवेदन आज सरकारी वकिललाई बुझाइँदै (score:0.8713335)

Recommended : १५ हजार घुस लिँदै गर्दा मालपोत बाराका खरदार र लेखापढी व्यवसायी पक्राउ (score:0.8681655)

Recommended : बालिका बलात्कार अभियोग लागेका अनाथालय प्रमुख थुनामा पठाइए (score:0.8657491)

Recommended : ढोरपाटनका मेयरलाई एमालेले गर्‍यो प्रदेश कमिटीबाट निलम्बन (score:0.86117494)

Recommended : सिम्रौनगढका तत्कालीन प्रमुख प्रशासकीय अधिकृतसहित ३ जनाविरुद्ध भ्रष्टाचार मुद्दा दायर (score:0.8611352

In [25]:
def recomendation(idx,no_of_news_article):
    #get similarity values with other articles
    similarity_score = list(enumerate(cosine_similarities[idx]))
    similarity_score = sorted(similarity_score, key=lambda x: x[1], reverse=True)
    # Get the scores of the n most similar news articles. Ignore the first movie.
    similarity_score = similarity_score[1:no_of_news_article+1]
    
    print("Article Read -- " + df['title'].iloc[idx] +" link --"+ df['link'].iloc[idx])
    print(" ---------------------------------------------------------- ")
    news_indices = [i[0] for i in similarity_score]
    for i in range(len(news_indices)):
        print("Recomendation "+ str(i+1)+" --- " +str(news_indices[i])+"(IDX)  "+ df['title'].iloc[news_indices[i]] +" || Link --"+ df['link'].iloc[news_indices[i]] +" score -- "+ str(similarity_score[i][1]))
        print()

In [27]:
idx=10
no_of_news_article=5
recomendation(idx,no_of_news_article)

Article Read -- जनकपुर ११ का वडाध्यक्ष र वडासचिवविरुद्ध भ्रष्टाचारको मुद्दा link --https://www.onlinekhabar.com/2023/05/1312674
 ---------------------------------------------------------- 
Recomendation 1 --- 2368(IDX)  शुद्धोधन-७ का वडा अध्यक्ष ५० हजार घुससहित पक्राउ || Link --https://www.onlinekhabar.com/2023/03/1284781 score -- 0.8842877

Recomendation 2 --- 1982(IDX)  नक्कली परीक्षार्थी राखेर एसईई दिएको आरोपमा मेयरविरुद्ध पक्राउ पुर्जी || Link --https://www.onlinekhabar.com/2023/04/1289031 score -- 0.88426626

Recomendation 3 --- 2169(IDX)  वृद्धभत्ताको रकम हिनामिनाको आरोपमा वडासचिव विरुद्ध भ्रष्टाचार मुद्दा || Link --https://www.onlinekhabar.com/2023/04/1286979 score -- 0.87886274

Recomendation 4 --- 3779(IDX)  सप्तरीको शम्भुनाथ नगरपालिकाका मेयरविरुद्ध भ्रष्टाचार मुद्दा || Link --https://www.onlinekhabar.com/2023/03/1270715 score -- 0.8761473

Recomendation 5 --- 235(IDX)  नक्कली भुटानी शरणार्थी प्रकरणको अनुसन्धान प्रतिवेदन आज सरकारी वकिललाई बुझाइँदै || Link --https://www.onlinek